In [1]:
import os
import sys
import cortex
import re
import numpy as np
import random
import time
import matplotlib.pyplot as plt
import pandas as pd
import torch
import seaborn as sns
import nibabel as nib
import pickle
import torchvision.models as models
import nibabel as nib
import h5py
import copy
import scipy.stats.mstats as mstats
import matplotlib.patches as patches
from PIL import Image
import argparse


print(sys.path)
%pwd

os.chdir('/home/rfpred')
sys.path.append('/home/rfpred/')
sys.path.append('/home/rfpred/envs/rfenv/lib/python3.11/site-packages/')
sys.path.append('/home/rfpred/envs/rfenv/lib/python3.11/site-packages/nsdcode')

from classes.regdata import RegData
from funcs.reloads import Reloader
from classes.natspatpred import NatSpatPred
from classes.voxelsieve import VoxelSieve
from unet_recon.inpainting import UNet

from funcs.rf_tools import make_circle_mask
from funcs.imgproc import get_bounding_box


['/home/rfpred/notebooks/construction_work', '/home/rfpred/envs/rfenv/lib/python311.zip', '/home/rfpred/envs/rfenv/lib/python3.11', '/home/rfpred/envs/rfenv/lib/python3.11/lib-dynload', '', '/home/rfpred/envs/rfenv/lib/python3.11/site-packages', '/home/rfpred/notebooks/alien_nbs/lgnpy', '/home/rfpred/envs/rfenv/lib/python3.11/site-packages/nsd_access-0.0.1.dev0-py3.11.egg', '/home/rfpred/envs/rfenv/lib/python3.11/site-packages/cifti-1.1-py3.11.egg']


In [2]:
NSP = NatSpatPred()
NSP.initialise(verbose=True)
rl = Reloader()

rois, roi_masks, viscortex_masks = NSP.cortex.visrois_dict(verbose=False)
prf_dict = NSP.cortex.prf_dict(rois, roi_masks)

Naturalistic Spatial Prediction class: Initialised

Class contains the following attributes:
 .analyse
 .attributes
 .cortex
 .datafetch
 .explore
 .hidden_methods
 .initialise
 .nsd_datapath
 .own_datapath
 .stimuli
 .subjects
 .utils


In [3]:
# Store the dense layer predictability estimate features

dense_preds = NSP.datafetch.store_predestims(cnn_type="vggfull",
                               peripheral=False,
                               dense=True,
                               )

In [5]:
dense_preds

,content_loss_0_L1,content_loss_0_MSE,content_loss_1_L1,content_loss_1_MSE,content_loss_2_L1,content_loss_2_MSE,img_ices,pixel_loss_L1,pixel_loss_MSE,pixel_loss_hole_L1,pixel_loss_hole_MSE,ssim,style_loss_0_L1,style_loss_0_MSE,style_loss_1_L1,style_loss_1_MSE,style_loss_2_L1,style_loss_2_MSE
0,0.047906,0.011707,0.076979,0.011961,0.408363,0.261266,0,0.095558,0.044879,0.095558,0.044879,0.557447,0.007432,0.000058,0.00207,0.000007,0.090966,0.011115
1,0.039105,0.006299,0.051831,0.006931,0.298033,0.14532,1,0.017005,0.001353,0.017005,0.001353,0.786186,0.001647,0.000004,0.002383,0.000008,0.054573,0.004677
2,0.033802,0.007621,0.036514,0.004088,0.219422,0.073495,2,0.103736,0.031104,0.103736,0.031104,0.325124,0.003937,0.00002,0.002717,0.000011,0.041093,0.002651
3,0.03546,0.007452,0.040793,0.004732,0.253758,0.097017,3,0.07244,0.013667,0.07244,0.013667,0.394794,0.003859,0.000018,0.002269,0.000008,0.048273,0.003626
4,0.034036,0.008969,0.047794,0.005624,0.247481,0.100965,4,0.114493,0.034366,0.114493,0.034366,0.356545,0.006752,0.000048,0.005148,0.000029,0.058878,0.005556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72995,0.043214,0.007001,0.052334,0.006955,0.304108,0.144095,72995,0.040269,0.006133,0.040269,0.006133,0.685852,0.003111,0.000012,0.004122,0.000019,0.057203,0.005034
72996,0.035877,0.008327,0.040559,0.004576,0.207223,0.066966,72996,0.118521,0.038764,0.118521,0.038764,0.329925,0.006793,0.00005,0.003566,0.000016,0.039181,0.002573
72997,0.031121,0.005193,0.035581,0.00362,0.195705,0.062442,72997,0.042956,0.005925,0.042956,0.005925,0.493293,0.003049,0.000013,0.003346,0.000015,0.073903,0.007812
72998,0.041438,0.006396,0.039441,0.005266,0.315916,0.157187,72998,0.024313,0.002335,0.024313,0.002335,0.611217,0.003541,0.000016,0.002619,0.000009,0.058458,0.005178


In [13]:
rl = Reloader()
NSP = rl.nsp()
NSP.initialise(verbose=True)

Naturalistic Spatial Prediction class: Initialised

Class contains the following attributes:
 .analyse
 .attributes
 .attributes_unfiltered
 .cortex
 .datafetch
 .explore
 .hidden_methods
 .initialise
 .nsd_datapath
 .own_datapath
 .stimuli
 .subjects
 .utils


In [3]:
Xpred_dense = NSP.stimuli.unpred_feats(cnn_type="vggfull",
                                       content=True,
                                       style=False,
                                       ssim=False,
                                       pixel_loss=False,
                                       L1=False,
                                       MSE=True,
                                       verbose=True,
                                       outlier_sd_bound="auto",
                                       subject="subj01",
                                       dense=True)

Fetched file from: all_predestims_vggfull.csv
['content_loss_0_MSE', 'content_loss_1_MSE', 'content_loss_2_MSE']


In [4]:
Xpred_dense.shape

(30000, 3)

In [6]:

# This is for the convolutional layers
Xpred_conv = NSP.stimuli.unpred_feats(
    cnn_type="vggfull",
    content=True,
    style=False,
    ssim=False,
    pixel_loss=False,
    L1=False,
    MSE=True,
    verbose=True,
    outlier_sd_bound=5,
    subject="subj01",
)

# This is for the dense layers
Xpred_dense = NSP.stimuli.unpred_feats(
    cnn_type="vggfull",
    content=True,
    style=False,
    ssim=False,
    pixel_loss=False,
    L1=False,
    MSE=True,
    verbose=True,
    outlier_sd_bound=5,
    subject="subj01",
    dense=True,
)
# wait untill all are computed of alexnet

Xpred = np.hstack((Xpred_conv, Xpred_dense))
n_layers = Xpred.shape[1]
print(f"Xpred has these dimensions: {Xpred.shape}")


Fetched file from: all_predestims_vggfull.csv
['content_loss_0_MSE', 'content_loss_1_MSE', 'content_loss_2_MSE', 'content_loss_3_MSE', 'content_loss_4_MSE', 'content_loss_5_MSE', 'content_loss_6_MSE', 'content_loss_7_MSE', 'content_loss_8_MSE', 'content_loss_9_MSE', 'content_loss_10_MSE', 'content_loss_11_MSE', 'content_loss_12_MSE', 'content_loss_13_MSE']
Fetched file from: all_predestims_vggfull.csv
['content_loss_0_MSE', 'content_loss_1_MSE', 'content_loss_2_MSE']
Xpred has these dimensions: (30000, 17)


In [10]:
print(Xpred_conv.shape[1])

for a in range(14, Xpred.shape[1]):
    print(a)

14
14
15
16


In [3]:
max_size = 2
min_size = 0.15
patchbound = 1
min_nsd_R2 = 0
min_prf_R2 = 0
peri_angles = [90, 210, 330]
angle = peri_angles[0]
peri_ecc = 2.0
startimg = 0
endimg = 10
# fixed_n_voxels = 50

In [4]:
# This voxeldict is not really needed, but I use it to get the exact matching
# mask for the peripheral patch
voxeldict = {}
print(f"Now working on patch with angle {angle}")
for roi in rois:
    print_attr = True if roi == rois[len(rois) - 1] else False
    voxeldict[roi] = VoxelSieve(
        NSP,
        prf_dict,
        roi_masks,
        subject="subj01",
        roi=roi,
        patchloc="peripheral",
        max_size=max_size,
        min_size=min_size,
        patchbound=patchbound,
        min_nsd_R2=min_nsd_R2,
        min_prf_R2=min_prf_R2,
        print_attributes=False,  # print_attr,
        fixed_n_voxels=None,
        peripheral_center=None,
        peri_angle=angle,
        peri_ecc=peri_ecc,
        leniency=0,
        verbose=False,
    )

mask1 = voxeldict[roi].patchmask


Now working on patch with angle 90
V1 voxels that fulfill requirements: 47 out of 7887.
V2 voxels that fulfill requirements: 52 out of 8296.
V3 voxels that fulfill requirements: 7 out of 7022.
V4 voxels that fulfill requirements: 11 out of 3976.


In [5]:
n_imgs = endimg - startimg
print(f"Processing {n_imgs} images, going from {startimg} to {endimg}")
print(f"Patch eccentricity: {peri_ecc}, patch angle: {angle}")
select_ices = list(range(startimg, endimg))

Processing 10 images, going from 0 to 10
Patch eccentricity: 2.0, patch angle: 90


In [6]:
# THIS BELOW IS FROM THE GET_PRED.PY SCRIPT
def draw_circmask(dims, maskrad, offset=(0,0), invert=True):
    import numpy as np
    y, x = np.ogrid[:dims[0], :dims[1]]
    center_x, center_y = dims[1] // 2 + offset[0], dims[0] // 2 + offset[1]
    mask = (x - center_x)**2 + (y - center_y)**2 <= maskrad**2
    return ~mask if invert else mask

def rand_img_list(n_imgs, asPIL:bool = True, add_masks:bool = True, mask_loc: str|np.ndarray = 'center', ecc_max = 1, select_ices = None, in_3d:bool = False):
    imgs = []
    img_nos = []
    for i in range(n_imgs):
        img_no = random.randint(0, 27999)
        if select_ices is not None:
            img_no = select_ices[i]
        # img = show_stim(img_no = img_no, hide = 'y')[0]
        img = NSP.stimuli.show_stim(img_no = img_no, hide=True, small=False, crop=False)[0]

        if i == 0:
            dim = img.shape[0]
            radius = ecc_max * (dim / 8.4)

            if type(mask_loc) == str:
                if mask_loc == 'center':
                    x = y = (dim + 1)/2
                elif mask_loc == 'irrelevant_patch':
                    x = y = radius + 10
            elif type(mask_loc) == np.ndarray:
                bounds = NSP.utils.get_bounding_box(mask_loc)
                patch_rad = bounds[1] - bounds[0]
                print(patch_rad)
                x = bounds[0] + patch_rad/2
                y = bounds[2] + patch_rad/2
                
        if asPIL:
            img = Image.fromarray(img)

        imgs.append(img)
        # img_nos.append(Image.fromarray(img_no))
        img_nos.append(img_no)
    mask = (make_circle_mask(dim, x, y, radius, fill = 'y', margin_width = 0) == 0)
    
    if in_3d:
        mask = _make_img_3d(mask)
    if asPIL:
        mask = Image.fromarray(mask)


    masks = [mask] * n_imgs
    
    if add_masks:
        return imgs, masks, img_nos
    else:
        return imgs, img_nos

def slice_array_with_mask(arr_in, mask_in):
    """
    Slices a 2D array using a 2D boolean mask with a contiguous square of True values.

    :param arr_in: 2D numpy array.
    :param mask_in: 2D boolean numpy array of the same shape as arr_in.
    :return: Sliced section of arr_in corresponding to the True values in mask_in.
    """
    # Find the indices of the mask where the value is True
    rows, cols = np.where(mask_in)
    top_left = (min(rows), min(cols))
    bottom_right = (max(rows), max(cols))

    # Slice the array
    return arr_in[top_left[0]:bottom_right[0]+1, top_left[1]:bottom_right[1]+1]

def scale_square_mask(mask_in:np.ndarray, scale_fact=np.sqrt(1.5), mask_val=1, min_size=50):
    """given a square mask, scale width and height with a given factor

    in:
    - mask_in: ndarray, (2d or 3d)
        boolean-type mask image
    - mask_val: float/int/bool (default:1)
        the value to look for as the definition of in the circle of the mask.
    - min_size: int
        minimum size of the square mask.

    out:
    -scaled_mask: ndarray
        like the square input mask, but now with a square outline around the mask
    """
    def _do_scaling(_mask_in:np.ndarray, scale_fact=np.sqrt(2), mask_val=1, min_size=50):
        """inner function doing the actual scaling"""
        mask_out=copy.deepcopy(_mask_in)
        nz_rows,nz_cols=np.nonzero(_mask_in==mask_val)
        nz_r,nz_c=np.unique(nz_rows),np.unique(nz_cols)
        # determine square masks that spans the circle
        width, height = nz_r[-1]-nz_r[0], nz_c[-1]-nz_c[0]

        # make actual spanning mask a bit larger (delta determined by scale_fact or min_size)
        ideal_delta_w = max(np.round(((width*scale_fact) - width)*.5), (min_size - width) // 2)
        ideal_delta_h = max(np.round(((height*scale_fact) - height)*.5), (min_size - height) // 2)

        # Adjust deltas based on mask's proximity to image borders
        delta_w_left = min(ideal_delta_w, nz_c[0])
        delta_w_right = min(ideal_delta_w, mask_out.shape[1] - nz_c[-1] - 1)
        delta_h_top = min(ideal_delta_h, nz_r[0])
        delta_h_bottom = min(ideal_delta_h, mask_out.shape[0] - nz_r[-1] - 1)

        # If mask is near the border, expand on the other side
        if delta_w_left < ideal_delta_w:
            delta_w_right = max(ideal_delta_w * 2 - delta_w_left, delta_w_right)
        if delta_w_right < ideal_delta_w:
            delta_w_left = max(ideal_delta_w * 2 - delta_w_right, delta_w_left)
        if delta_h_top < ideal_delta_h:
            delta_h_bottom = max(ideal_delta_h * 2 - delta_h_top, delta_h_bottom)
        if delta_h_bottom < ideal_delta_h:
            delta_h_top = max(ideal_delta_h * 2 - delta_h_bottom, delta_h_top)

        mask_out[int(nz_r[0]-delta_h_top):int(nz_r[-1]+delta_h_bottom),
                 int(nz_c[0]-delta_w_left):int(nz_c[-1]+delta_w_right)] = mask_val
        # set values to 1, square mask
        return(mask_out)

    # switch dealing with RGB [colmns,rows,colours] vs grayscale images [columns,rows]
    if mask_in.ndim==3:
        mask_scaled=_do_scaling(mask_in[:,:,0],scale_fact=scale_fact, mask_val=mask_val, min_size=min_size)
        return(_make_img_3d(mask_scaled))
    elif mask_in.ndim==2:
        return(_do_scaling(mask_in, scale_fact=scale_fact, mask_val=mask_val, min_size=min_size))
    else:
        raise ValueError('can only understand 3d (RGB) or 2d array images!')

def _make_img_3d(mask_in,):
    """for 2d array, copy to make 3-dimensional"""
    return(np.repeat(mask_in[:,:,np.newaxis],3,axis=2))




In [7]:
mask_radius=100
rf_mask=draw_circmask((425,425),mask_radius)

In [12]:
# unet=UNet(checkpoint_name='pconv_circ-places20k.pth',feature_model='alex')
# unet=UNet(checkpoint_name='pconv_circ-places20k.pth',feature_model='vgg-conv')
# unet=UNet(checkpoint_name='pconv_circ-places20k.pth',feature_model='vgg-conv-dense')
unet=UNet(checkpoint_name='pconv_circ-places20k.pth',feature_model='vgg-dense')

imgs, masks, img_nos = rand_img_list(n_imgs, asPIL = True, add_masks = True, mask_loc = mask1, ecc_max = 1, select_ices = select_ices, in_3d = False)

# from scipy.ndimage import zoom

# # Define the new size
# new_size = (224, 224)

# # Calculate the scaling factors
# y_scale = new_size[0] / mask1.shape[0]
# x_scale = new_size[1] / mask1.shape[1]

# # Resize the array
# rf_mask_in = zoom(mask1, (y_scale, x_scale))


rf_mask_in = mask1
rf_mask_nsd = rf_mask_in == 0
xmin,xmax,ymin,ymax = list(get_bounding_box(rf_mask_in))
crop_mask = rf_mask_in[ymin:ymax, xmin:xmax] == 1

# THIS IS THE ORIGINAL ONE, THE CORRECT CROP
# eval_fact=np.sqrt(1.2) # This needs to be in correspondence with the min_size (original eval_fact = 1.5, min_size = 100)
eval_fact=np.sqrt(1) # This needs to be in correspondence with the min_size (original eval_fact = 1.5, min_size = 100)

# THIS IS THE FULL IMG FEATUREMAP EVALMASK
# eval_fact=np.sqrt(18)
eval_mask=scale_square_mask(~np.array(masks[0]), min_size=80, scale_fact= eval_fact)

Loading the Model...
Running the inpainting file from /rfpred/src/inpainting.py
100


In [13]:
# from PIL import Image

# # Define the new size
# new_size = (224, 224)

# # Resize the images
# imgs = [img.resize(new_size) for img in imgs]

# # Resize the masks
# masks = [mask.resize(new_size) for mask in masks]

In [17]:
(10 * 2 * 73)/60

# so it will take about 3 days to compute all the dense layers, ight

24.333333333333332

In [14]:
start_time = time.time()
 
# Run them through the U-Net
# payload_nsd = unet.analyse_images(imgs, masks, return_recons=True, eval_mask = None)
payload_nsd_crop = unet.analyse_images(imgs, masks, return_recons=True, eval_mask = eval_mask)

end_time = time.time()

total_time = end_time - start_time
average_time_per_image = (total_time / n_imgs) #/ 2
print(f'\nThis took {total_time} seconds, or {total_time / 60} minutes, or {total_time / 3600} hours')
print(f"Average time per image: {average_time_per_image} seconds\n")

# Add the specific image indices to the dictionaries. 
payload_nsd_crop['img_ices'] = img_nos

excl = ['recon_dict']
payload_light = {k: v for k, v in payload_nsd_crop.items() if k not in excl}

print("succeeded")

torch.Size([10, 512, 60, 60])
torch.Size([10, 512, 7, 7])
torch.Size([10, 25088])
torch.Size([10, 512, 60, 60])
torch.Size([10, 512, 7, 7])
torch.Size([10, 25088])
torch.Size([10, 512, 60, 60])
torch.Size([10, 512, 7, 7])
torch.Size([10, 25088])
torch.Size([10, 512, 60, 60])
torch.Size([10, 512, 7, 7])
torch.Size([10, 25088])

This took 5.104436635971069 seconds, or 0.08507394393285116 minutes, or 0.0014178990655475193 hours
Average time per image: 0.5104436635971069 seconds

succeeded


In [15]:
payload_light

{'ssim': array([0.77164, 0.72865, 0.44022, 0.28945, 0.36729, 0.19352, 0.23294,
        0.76495, 0.20038, 0.37313]),
 'pixel_loss_L1': array([0.01383, 0.02451, 0.05972, 0.10519, 0.12101, 0.14505, 0.15693,
        0.01424, 0.11911, 0.09799], dtype=float32),
 'content_loss_0_L1': array([0.05862, 0.05189, 0.04057, 0.03755, 0.04388, 0.03282, 0.05116,
        0.05156, 0.05606, 0.03804], dtype=float32),
 'content_loss_1_L1': array([0.05382, 0.05437, 0.04589, 0.04857, 0.05368, 0.0344 , 0.05961,
        0.04794, 0.07418, 0.04461], dtype=float32),
 'content_loss_2_L1': array([0.34303, 0.31337, 0.25525, 0.23361, 0.26423, 0.168  , 0.34325,
        0.3587 , 0.38128, 0.21495], dtype=float32),
 'style_loss_0_L1': array([0.00384, 0.00348, 0.00698, 0.00594, 0.00685, 0.00571, 0.00718,
        0.00368, 0.00786, 0.00576], dtype=float32),
 'style_loss_1_L1': array([0.00691, 0.0066 , 0.00595, 0.00413, 0.00644, 0.00334, 0.00425,
        0.00554, 0.00511, 0.00385], dtype=float32),
 'style_loss_2_L1': array([0

In [14]:
payload_light

{'ssim': array([0.77164, 0.72865, 0.44022, 0.28945, 0.36729, 0.19352, 0.23294,
        0.76495, 0.20038, 0.37313]),
 'pixel_loss_L1': array([0.01383, 0.02451, 0.05972, 0.10519, 0.12101, 0.14505, 0.15693,
        0.01424, 0.11911, 0.09799], dtype=float32),
 'content_loss_0_L1': array([ 2285.5752, 12772.386 , 24303.463 , 33766.188 , 57594.062 ,
        42056.703 , 90708.3   ,  3386.5386, 28515.32  , 43089.89  ],
       dtype=float32),
 'content_loss_1_L1': array([ 24206.162, 119084.164,  90144.734, 173476.97 , 306038.72 ,
        177473.69 , 318200.3  ,  17460.734, 110202.97 ,  91451.305],
       dtype=float32),
 'content_loss_2_L1': array([ 89624.805, 381693.2  , 272676.44 , 573994.2  , 994592.7  ,
        654986.44 , 944356.25 ,  61283.465, 293572.   , 401516.4  ],
       dtype=float32),
 'style_loss_0_L1': array([2.56114e+11, 2.00052e+11, 6.20967e+11, 4.47141e+11, 5.27445e+11,
        7.06104e+11, 1.47407e+12, 2.62729e+11, 5.43489e+11, 1.07795e+12],
       dtype=float32),
 'style_loss